In [ ]:
%matplotlib inline
import nest_asyncio
nest_asyncio.apply()

import multiprocessing
multiprocessing.set_start_method("fork")

import pickle
import os
import math
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import random
import seaborn as sns
import math
import statsmodels
import stan
import arviz as az
import cmdstanpy
from cmdstanpy import cmdstan_path, CmdStanModel

In [ ]:
df = pd.read_pickle("v33_real_data.pkl")

In [ ]:
df.groupby(['country_number'])['week'].count()

In [ ]:
# Number of countries
totalN = len(df)
J = len(df.country_number.unique())
S = len(df.site_number.unique())
country = np.reshape((np.array([df.country_number])),(totalN,))
site = (np.reshape((np.array([df.site_number])),(totalN,)))
N = list(df.groupby(['country_number']).count().max(axis=1))
maxN = max(df.groupby(['country_number']).count().max(axis=1))
country_of_site = np.array(df.groupby(['country_number','site_number']).size().reset_index()['country_number'])
site_df = df.groupby(['site_number'])['week'].count()
site_df = pd.DataFrame([site_df]).T

In [ ]:
df[df['country_number']==0]
dfs_to_join_x = []
dfs_to_join_y = []
dfs_to_join_s = []

for j in range(J):
    temp_df = df[df['country_number']==j+1]
    temp_df_x = pd.DataFrame(temp_df['week'])
    temp_df_y = pd.DataFrame(temp_df['number_recruited'])
    temp_df_s = pd.DataFrame(temp_df['site_number'])

    temp_df_x = temp_df_x.reset_index(drop=True)
    temp_df_y = temp_df_y.reset_index(drop=True)
    temp_df_s = temp_df_s.reset_index(drop=True)

    dfs_to_join_x.append(temp_df_x)
    dfs_to_join_y.append(temp_df_y)
    dfs_to_join_s.append(temp_df_s)

df_to_join_x = pd.concat(dfs_to_join_x, axis=1)
df_to_join_y = pd.concat(dfs_to_join_y, axis=1)
df_to_join_s = pd.concat(dfs_to_join_s, axis=1)


df_to_join_x = df_to_join_x.fillna(0)
df_to_join_y = df_to_join_y.fillna(0)
df_to_join_s = df_to_join_s.fillna(0)

y = np.array(df_to_join_y.T).astype(int)
x = np.array(df_to_join_x.T)
site = np.array(df_to_join_s.T).astype(int)
n_tilde = 200

In [ ]:
x_tilde = np.zeros((J,n_tilde))
for i in range(J):
    x_tilde[i] = np.linspace(1, np.array(df.groupby(['country_number'], sort=False)['week'].max())[i]+5, n_tilde)

In [ ]:
real_data = {'J': J, 'maxN': maxN, 'N': N, 'y': y, 'x': x, 'S':S, 'site':site, 'n_tilde':n_tilde, 'country_of_site':country_of_site}

# 1. Homogeneous Poisson-Gamma

In [ ]:
with open('real_hpg.pickle', 'rb') as f:
    hpg_list = pickle.load(f)
r_hpg_model, r_hpg_fit = hpg_list

In [ ]:
hpg_arviz = az.from_cmdstanpy(
    posterior= r_hpg_fit,
    observed_data={"y": y},
    log_likelihood="log_lik"
)

### Diagnostics

In [ ]:
az.plot_posterior(hpg_arviz, var_names=['lambda'])

In [ ]:
az.plot_trace(hpg_arviz, var_names=['lambda'])

In [ ]:
az.ess(hpg_arviz, var_names=['lambda'])

In [ ]:
az.rhat(hpg_arviz, var_names=['lambda'])

# 2. Hierarchical Homogeneous Poisson-Gamma

In [ ]:
with open('real_hhpg.pickle', 'rb') as f:
    hhpg_list = pickle.load(f)
r_hhpg_model, r_hhpg_fit = hhpg_list

In [ ]:
hhpg_arviz = az.from_cmdstanpy(
    posterior= r_hhpg_fit,
    observed_data={"y": y},
    log_likelihood="log_lik"
)

### Diagnostics

In [ ]:
az.plot_posterior(hhpg_arviz, var_names=['site_lambda'])

In [ ]:
az.plot_trace(hhpg_arviz, var_names=['site_lambda'])

In [ ]:
az.ess(hhpg_arviz, var_names=['site_lambda'])

In [ ]:
az.rhat(hhpg_arviz, var_names=['site_lambda'])

# 3. Conditional Linear Model

In [ ]:
with open('real_clm.pickle', 'rb') as f:
    clm_list = pickle.load(f)
r_clm_model, r_clm_fit = clm_list

In [ ]:
clm_arviz = az.from_cmdstanpy(
    posterior= r_clm_fit,
    observed_data={"y": y},
    log_likelihood="log_lik"
)

### Diagnostics

In [ ]:
az.plot_posterior(clm_arviz, var_names=['country_a','country_b'])

In [ ]:
az.plot_trace(clm_arviz, var_names=['country_a','country_b'])

In [ ]:
az.ess(clm_arviz, var_names=['country_a','country_b'])

In [ ]:
az.rhat(clm_arviz, var_names=['country_a','country_b'])

# 4. Hierarchical Condtional Linear

In [ ]:
with open('real_hclm.pickle', 'rb') as f:
    hclm_list = pickle.load(f)
r_hclm_model, r_hclm_fit = hclm_list

In [ ]:
hclm_arviz = az.from_cmdstanpy(
    posterior= r_hclm_fit,
    observed_data={"y": y},
    log_likelihood="log_lik"
)

### Diagnostics

In [ ]:
az.plot_posterior(hclm_arviz, var_names=['site_a','site_b'])

In [ ]:
az.plot_trace(hclm_arviz, var_names=['site_a','site_b'])

In [ ]:
az.ess(hclm_arviz, var_names=['site_a','site_b'])

In [ ]:
az.rhat(hclm_arviz, var_names=['site_a','site_b'])

# 5. LGCP

In [ ]:
with open('real_non_hierarchical_GP_prior2.pickle', 'rb') as f:
    lgcp_list = pickle.load(f)
r_lgcp_model, r_lgcp_fit = lgcp_list

In [ ]:
lgcp_arviz = az.from_cmdstanpy(
    posterior= r_lgcp_fit,
    observed_data={"y": y},
    log_likelihood="log_lik"
)

### Diagnostics

In [ ]:
az.plot_posterior(lgcp_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])

In [ ]:
az.plot_trace(lgcp_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])

In [ ]:
az.ess(lgcp_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])

In [ ]:
az.rhat(lgcp_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])

# 6. HLGCP - Prior 1

In [ ]:
with open('real_hierarchical_GP_prior2.pickle', 'rb') as f:
    hlgcp1_list = pickle.load(f)
r_hlgcp1_model, r_hlgcp1_fit = hlgcp1_list

In [ ]:
hlgcp1_arviz = az.from_cmdstanpy(
    posterior= r_hlgcp1_fit,
    observed_data={"y": y},
    log_likelihood="log_lik"
)

### Diagnostics

In [ ]:
az.plot_posterior(hlgcp_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])

In [ ]:
az.plot_trace(hlgcp_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])

In [ ]:
az.ess(hlgcp_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])

In [ ]:
az.rhat(hlgcp_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])

# 7. HLGCP - Prior 2

In [ ]:
with open('real_hierarchical_GP_prior3.pickle', 'rb') as f:
    hlgcp2_list = pickle.load(f)
r_hlgcp2_model, r_hlgcp2_fit = hlgcp2_list

In [ ]:
hlgcp2_arviz = az.from_cmdstanpy(
    posterior= r_hlgcp2_fit,
    observed_data={"y": y},
    log_likelihood="log_lik"
)

### Diagnostics

In [ ]:
az.plot_posterior(hlgcp2_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])

In [ ]:
az.plot_trace(hlgcp2_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])

In [ ]:
az.ess(hlgcp2_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])

In [ ]:
az.rhat(hlgcp2_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])

# 8. HLGCP - Prior 3

In [ ]:
with open('real_hierarchical_GP_prior4.pickle', 'rb') as f:
    hlgcp_list3 = pickle.load(f)
r_hlgcp3_model, r_hlgcp3_fit = hlgcp_list3

In [ ]:
hlgcp3_arviz = az.from_cmdstanpy(
    posterior=r_hlgcp3_fit,
    observed_data={"y": y},
    log_likelihood="log_lik"
)

### Diagnostics

In [ ]:
az.plot_posterior(hlgcp3_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])


In [ ]:
az.plot_trace(hlgcp3_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])


In [ ]:
az.ess(hlgcp3_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])

In [ ]:
az.rhat(hlgcp3_arviz, var_names=['country_a', 'country_rho', 'country_alpha'])


# 9. HLGCPT

In [ ]:
with open('real_hier_GP_trend.pickle', 'rb') as f:
    hlgcpt_list = pickle.load(f)
r_hlgcpt_fit, r_hlgcpt_fit = hlgcpt_list

In [ ]:
hlgcpt_arviz = az.from_cmdstanpy(
    posterior=r_hlgcpt_fit,
    observed_data={"y": y},
    log_likelihood="log_lik"
)

### Diagnostics

In [ ]:
az.plot_posterior(hlgcpt_arviz, var_names=['country_a', 'country_b','country_rho', 'country_alpha'])

In [ ]:
az.plot_trace(hlgcpt_arviz, var_names=['country_a','country_b', 'country_rho', 'country_alpha'])

In [ ]:
az.ess(hlgcpt_arviz, var_names=['country_a','country_b', 'country_rho', 'country_alpha'])

In [ ]:
az.rhat(hlgcpt_arviz, var_names=['country_a','country_b', 'country_rho', 'country_alpha'])

# 10. HSGCP

In [ ]:
with open('real_hierarchical_sigmoidal.pickle', 'rb') as f:
    hsgcp_list = pickle.load(f)
r_hsgcp_model, r_hsgcp_fit = hsgcp_list

In [ ]:
hsgcp_arviz = az.from_cmdstanpy(
    posterior=r_hsgcp_fit,
    observed_data={"y": y},
    log_likelihood="log_lik"
)

### Diagnostics

In [ ]:
az.plot_posterior(hsgcp_arviz, var_names=['country_a', 'country_rho', 'country_alpha', 'site_lambda'])

In [ ]:
az.plot_trace(hsgcp_arviz, var_names=['country_a', 'country_rho', 'country_alpha', 'site_lambda'])

In [ ]:
az.ess(hsgcp_arviz, var_names=['country_a', 'country_rho', 'country_alpha', 'site_lambda'])

In [ ]:
az.rhat(hsgcp_arviz, var_names=['country_a', 'country_rho', 'country_alpha', 'site_lambda'])

## Posterior Functions of Baseline models

In [ ]:
# To plot posterior predictive plots for hierarchical
func_mat1 = np.zeros((2,1000,11,200))
for i in range(1000): # no. of chains
    for j in range(11): # no. of countries
        for k in range(200):
            func_mat1[0,i,j,k] = hpg_arviz['posterior']['lambda'][0,i,j]
            
# To plot posterior predictive plots for hierarchical
for i in range(1000): # no. of chains
    for j in range(11): # no. of countries
        for k in range(200):
            func_mat1[1,i,j,:] = np.exp(np.array(clm_arviz['posterior']['country_a'][0,i,j]) +  np.array(clm_arviz['posterior']['country_b'][0,i,j])* x_tilde[j])
            

## Posterior Functions of LGCP, HLGCP and HLGCPT

In [ ]:
with open('lgcp_initial_prior_13.pickle', 'rb') as f:
    func_mat_lgcp = pickle.load(f)

In [ ]:
func_mat2 = np.zeros((5,1000,11,200))
for i in range(1000): # no. of chains
    for j in range(11): # no. of countries
        for k in range(200):
            func_mat2[0,i,j,k] = lgcp_arviz['posterior']['a_plus_f'][0,i,j+11*(k)]
            
for i in range(1000): # no. of chains
    for j in range(11): # no. of countries
        for k in range(200):
            func_mat2[1,i,j,k] = hlgcp1_arviz['posterior']['a_plus_f'][0,i,j+11*(k)]

for i in range(1000): # no. of chains
    for j in range(11): # no. of countries
        for k in range(200):
            func_mat2[2,i,j,k] = hlgcp2_arviz['posterior']['a_plus_f'][0,i,j+11*(k)]
            
for i in range(1000): # no. of chains
    for j in range(11): # no. of countries
        for k in range(200):
            func_mat2[3,i,j,k] = hlgcp3_arviz['posterior']['a_plus_f'][0,i,j+11*(k)]
            

for i in range(1000): # no. of chains
    for j in range(11): # no. of countries
        for k in range(200):
            func_mat2[4,i,j,k] = hlgcpt_arviz['posterior']['a_plus_f'][0,i,j+11*(k)]
            

## Plots of Posterior Functions

In [ ]:
fig, axs = plt.subplots(11,2, figsize=(30,60),  constrained_layout=True)
fig.suptitle('Posterior Predictive Check Plots of LGCP')
name_list = ['Prior 1 ', 'Prior 2 ']
ddd_list = []

for k in range(2):
    if k == 0 : 
        count = -1
    for j in range(11):
        if j == 1:
            count += 1
        elif j == 7 :
            count += 1
        else : 
            pass
        count += 1
        ddd_list.append(count)

        for i in np.random.randint(0, 1000, 400):
            if k==0:
                axs[j,k].plot(x_tilde[j], func_mat_lgcp[i,count,:], color='lightsteelblue',
                        alpha = 0.5)
                
            elif k==1: 
                axs[j,k].plot(x_tilde[j], func_mat2[0,i,j,:], color='lightsteelblue',
                    alpha = 0.5)
            axs[j,k].set_title('LGCP-' + str(name_list[k]) + ' Country ' + str(j+1))

In [ ]:
model_list = ['Prior 1 ', 'Prior 2 ', 'Prior 3 ']

fig, axs = plt.subplots(11,3, figsize=(30,60),  constrained_layout=True)
fig.suptitle('Posterior Predictive Check Plots of HLGCP')
for k in range(3):
    for j in range(11):
        for i in np.random.randint(0, 1000, 400):
            if k==0: 
                axs[j,k].plot(x_tilde[j, :180], func_mat2[1,i,j,:180], color='lightsteelblue',
                alpha = 0.5)
            elif k==1: 
                axs[j,k].plot(x_tilde[j, :180], func_mat2[2,i,j,:180], color='lightsteelblue',
                alpha = 0.5)
            else :
                axs[j,k].plot(x_tilde[j, :180], func_mat2[3,i,j,:180], color='lightsteelblue',
                alpha = 0.5)
#         axs[j,k].plot(x_tilde[j], intensity[j], color='red')
        axs[j,k].set_title('HLGCP-' + str(model_list[k])+' Country ' + str(j+1))

In [ ]:
model_list = ['HPG','CLM']

fig, axs = plt.subplots(11,2, figsize=(30,60),  constrained_layout=True)
fig.suptitle('Posterior Predictive Check Plots of Baseline models')
for k in range(2):
    for j in range(11):
        for i in np.random.randint(0, 1000, 400):
            if k==0: 
                axs[j,k].plot(x_tilde[j], func_mat1[k,i,j,:], color='lightsteelblue',
                alpha = 0.5)
            elif k==1: 
                axs[j,k].plot(x_tilde[j], func_mat1[k,i,j,:], color='lightsteelblue',
                alpha = 0.5)
#             else :
#                 axs[j,k].plot(x_tilde[j], func_mat2[0,i,j,:], color='lightsteelblue',
#                 alpha = 0.5)
#         axs[j,k].plot(x_tilde[j], intensity[j], color='red')
        axs[j,k].set_title(str(model_list[k])+' Country ' + str(j+1))

In [ ]:
model_list = ['HLGCP', 'HLGCPT']

fig, axs = plt.subplots(11,2, figsize=(30,60),  constrained_layout=True)
fig.suptitle('Posterior Predictive Check Plots of Latent Functions')
for k in range(2):
    for j in range(11):
        for i in np.random.randint(0, 1000, 400):
            axs[j,k].plot(x_tilde[j, :180], func_mat2[k+3,i,j,:180], color='lightsteelblue',
                alpha = 0.5)
#         axs[j,k].plot(x_tilde[j], intensity[j], color='red')
        axs[j,k].set_title(str(model_list[k])+' Country ' + str(j+1))

# 11. HSGCPT

In [ ]:
with open('real_hierarchical_sigmoidal_trend.pickle', 'rb') as f:
    hsgcpt_list = pickle.load(f)
r_hsgcpt_model, r_hsgcpt_fit = hsgcpt_list

In [ ]:
hsgcpt_arviz = az.from_cmdstanpy(
    posterior=r_hsgcpt_fit,
    observed_data={"y": y},
    log_likelihood="log_lik"
)

### Diagnostics

In [ ]:
az.plot_posterior(hsgcpt_arviz, var_names=['country_a', 'country_rho', 'country_alpha', 'site_lambda'])

In [ ]:
az.plot_trace(hsgcpt_arviz, var_names=['country_a', 'country_rho', 'country_alpha', 'site_lambda'])

# Posterior Summary Statistics

In [ ]:
hpg_summary = az.summary(hpg_arviz, var_names = ['lambda'])[['mean','sd','hdi_3%','hdi_97%',
                                              'r_hat']]

In [ ]:
hpg_summary.reset_index(level=0, inplace=True)
print(hpg_summary.to_latex(index=False)) 

In [ ]:
hhpg_summary = az.summary(hhpg_arviz, var_names = ['site_lambda'])[
    ['mean','sd','hdi_3%','hdi_97%','r_hat']]

In [ ]:
hhpg_summary.reset_index(level=0, inplace=True)
hhpg_summary = pd.concat([hhpg_summary, site_df2], axis=1)[['index','week','mean','sd','hdi_3%','hdi_97%','r_hat']]
print(hhpg_summary.to_latex(index=False))  

In [ ]:
clm_summary = az.summary(clm_arviz, var_names = ['country_a','country_b'])[
    ['mean','sd','hdi_3%','hdi_97%','r_hat']]

In [ ]:
clm_summary.reset_index(level=0, inplace=True)
print(clm_summary.to_latex(index=False)) 

In [ ]:
hclm_summary = az.summary(hclm_arviz, var_names = ['site_a','site_b'])[
    ['mean','sd','hdi_3%','hdi_97%','r_hat']]

In [ ]:
site_df22 = pd.concat([site_df2]*2, ignore_index=True)

In [ ]:
hclm_summary.reset_index(level=0, inplace=True)
hclm_summary = pd.concat([hclm_summary, site_df22], axis=1)[['index','week','mean','sd','hdi_3%','hdi_97%','r_hat']]
print(hclm_summary.to_latex(index=False))  

In [ ]:
lgcp_summary = az.summary(lgcp_arviz, var_names = ['country_rho','country_alpha','country_a'])[
    ['mean','sd','hdi_3%','hdi_97%','r_hat']]

In [ ]:
lgcp_summary.reset_index(level=0, inplace=True)
print(lgcp_summary.to_latex(index=False)) 

In [ ]:
hlgcp1_summary = az.summary(hlgcp1_arviz, var_names = ['country_rho','country_alpha','country_a'])[
    ['mean','sd','hdi_3%','hdi_97%','r_hat']]

In [ ]:
hlgcp1_summary.reset_index(level=0, inplace=True)
print(hlgcp1_summary.to_latex(index=False))  

In [ ]:
hlgcp2_summary = az.summary(hlgcp2_arviz, var_names = ['country_rho','country_alpha','country_a'])[
    ['mean','sd','hdi_3%','hdi_97%','r_hat']]

In [ ]:
hlgcp2_summary.reset_index(level=0, inplace=True)
print(hlgcp2_summary.to_latex(index=False))  

In [ ]:
hlgcp3_summary = az.summary(hlgcp3_arviz, var_names = ['country_rho','country_alpha','country_a'])[
    ['mean','sd','hdi_3%','hdi_97%','r_hat']]

In [ ]:
hlgcp3_summary.reset_index(level=0, inplace=True)
print(hlgcp3_summary.to_latex(index=False))  

In [ ]:
hlgcpt_summary = az.summary(hlgcpt_arviz, var_names = ['country_rho','country_alpha','country_a','country_b'])[
    ['mean','sd','hdi_3%','hdi_97%','r_hat']]

In [ ]:
hlgcpt_summary.reset_index(level=0, inplace=True)
print(hlgcpt_summary.to_latex(index=False))  

In [ ]:
hsgcp_country_summary = az.summary(hsgcp_arviz, var_names = ['country_rho','country_alpha','country_a'])[
    ['mean','sd','hdi_3%','hdi_97%','r_hat']]

In [ ]:
hsgcp_country_summary.reset_index(level=0, inplace=True)
print(hsgcp_country_summary.to_latex(index=False))  

In [ ]:
hsgcp_site_summary = az.summary(hsgcp_arviz, var_names = ['site_lambda'])[
    ['mean','sd','hdi_3%','hdi_97%','r_hat']]

In [ ]:
hsgcp_site_summary.reset_index(level=0, inplace=True)
hsgcp_site_summary = pd.concat([hsgcp_site_summary, site_df2], axis=1)[['index','week','mean','sd','hdi_3%','hdi_97%','r_hat']]
print(hsgcp_site_summary.to_latex(index=False))  

In [ ]:
hsgcpt_country_summary = az.summary(hsgcpt_arviz, var_names = ['country_rho','country_alpha','country_a','country_b'])[
    ['mean','sd','hdi_3%','hdi_97%','r_hat']]

In [ ]:
hsgcpt_country_summary.reset_index(level=0, inplace=True)
print(hsgcpt_country_summary.to_latex(index=False))  

In [ ]:
hsgcpt_site_summary = az.summary(hsgcpt_arviz, var_names = ['site_lambda'])[
    ['mean','sd','hdi_3%','hdi_97%','r_hat']]

In [ ]:
hsgcpt_site_summary.reset_index(level=0, inplace=True)
hsgcpt_site_summary = pd.concat([hsgcpt_site_summary, site_df2], axis=1)[['index','week','mean','sd','hdi_3%','hdi_97%','r_hat']]

In [ ]:
# hsgcpt_site_summary['index'] = hsgcpt_site_summary['index'].shift(-1)
print(hsgcpt_site_summary.to_latex(index=False))  

## Model Comparison

In [ ]:
compare_dict = {
    "HPG":hpg_arviz, 
    "HHPG":hhpg_arviz, 
    "CLM":clm_arviz, 
    "HCLM":hclm_arviz, 
    "LGCP":lgcp_arviz, 
    "HLGCP-Prior 1":hlgcp1_arviz, 
    "HLGCP-Prior 2":hlgcp2_arviz, 
    "HLGCP-Prior 3":hlgcp3_arviz, 
    "HLGCPT":hlgcpt_arviz, 
    "HSGCP":hsgcp_arviz, 
    "HSGCPT":hsgcpt_arviz
}
df_comp_loo = az.compare(compare_dict)

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
df_comp_loo = df_comp_loo[['rank','loo','se','weight']]

In [ ]:
df_comp_loo.reset_index(level=0, inplace=True)
print(df_comp_loo.to_latex(index=False)) 